In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
df=pd.read_csv('data.csv')
import pickle

# Data Preprocessing

In [ ]:
data=df.to_numpy()
data1=data[:,0]
x=len(data1)
seperations=[0]
for i in range(len(data1)-1):
    if (data1[i+1]-data1[i])>=500:
        seperations.append(i)
    else:
        pass
seperations.append(x)
seperations

In [ ]:
# normalization
segments=[]
for i in range(len(seperations)-1):
    segments.append(data[(seperations[i]+3):(seperations[i+1]-3)])
len(segments)

In [ ]:
a=np.concatenate(segments)
plt.scatter(a[:,1],a[:,2])
type(segments)

# Rubine Feature Extraction

In [ ]:
def dx(i): return dat.x[i]-dat.x[i-1]
def dy(i): return dat.y[i]-dat.y[i-1]
def dt(i): return dat.time[i]-dat.time[i-1]
def theta(i):return math.atan2(dx(i+1)*dy(i)-dx(i)*dy(i+1),dx(i+1)*dx(i)+dy(i+1)*dy(i))
def sigma(start, end, fun): return sum([fun(i) for i in range(start, end)])
def rf1(d): return (d.x[2]-d.x[0])/math.sqrt((d.y[2]-d.y[0])**2+(d.x[2]-d.x[0])**2)
def rf2(d): return (d.y[2]-d.y[0])/math.sqrt((d.y[2]-d.y[0])**2+(d.x[2]-d.x[0])**2)
def rf3(d): return math.sqrt((max(d.y)-min(d.y))**2+(max(d.x)-min(d.x))**2)
def rf4(d): return math.atan2(max(d.y)-min(d.y), max(d.x)-min(d.x))
def rf5(d): return math.sqrt((d.x[len(d)-1]-d.x[0])**2+(d.y[len(d)-1]-d.y[0])**2)
def rf6(d): return (d.x[len(d)-1]-d.x[0])/rf5(d)
def rf7(d): return (d.y[len(d)-1]-d.y[0])/rf5(d)
def rf8(d): return sigma(1, len(d), lambda i: math.sqrt(dx(i)**2+dy(i)**2))
def rf9(d): return sigma(1, len(d)-1, theta)
def rf10(d): return sigma(1, len(d)-1, lambda i: abs(theta(i)))
def rf11(d): return sigma(1, len(d)-1, lambda i: theta(i)**2)
def rf12(d): return max([(dx(i)**2+dy(i)**2)/dt(i)**2 if dt(i) > 0 else 0 for i in range(1, len(d))])
def rf13(d): return d.time[len(d)-1]-d.time[0]

In [ ]:
df_list=[]
for i in segments:
    df_list.append(pd.DataFrame(i,columns=['time','x','y']))
stroke_features=[]
for i in range(len(df_list)):
    temp=[]
    dat=df_list[i]
    f1=rf1(df_list[i])
    f2=rf2(df_list[i])
    f3=rf3(df_list[i])
    f4=rf4(df_list[i])
    f5=rf5(df_list[i])
    f6=rf6(df_list[i])
    f7=rf7(df_list[i])
    f8=rf8(df_list[i])
    f9=rf9(df_list[i])
    f10=rf10(df_list[i])
    f11=rf11(df_list[i])
    f12=rf12(df_list[i])
    f13=rf13(df_list[i])
    temp+=[f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13]
    stroke_features.append(temp)

In [ ]:
df=pd.DataFrame(stroke_features,columns=['rf1','rf2','rf3','rf4','rf5','rf6','rf7','rf8','rf9','rf10','rf11','rf12','rf13'])
df

# Single Link Euclidean Distance between strokes

In [ ]:
def distance(u,v):
    minimum=[]
    min_d=[]
    u=np.array(u)
    v=np.array(v)
    u=np.delete(u,0,1)
    v=np.delete(v,0,1)
    for i in range(len(u)):
        for j in range(len(v)):
            d=math.dist(u[i],v[j])
            minimum.append(d)
        minimum.sort()
        minum=minimum[0]
        min_d.append(minum)
    min_d.sort()
    return min_d[0]
composite=[]
for i in range(len(segments)):
    un=segments[i]
    distarr=[]
    dist_index=[]
    for j in range(len(segments)):
        vn=segments[j]
        sldist=distance(un,vn)
        distarr.append(sldist)
        dist_index.append(j)
    matrix=np.array([dist_index,distarr])
    composite.append(matrix)
c=np.array(composite)
c

# Distinguishing between line and non-line segments

In [ ]:
line_arr=[]
obj_arr=[]
for i in range(len(stroke_features)):
    r=stroke_features[i]
    end_to_end=r[4]
    length=r[7]
    if end_to_end>=0.90*length:
        line_arr.append(i)
    else:
        obj_arr.append(i)
        pass
line_arr,obj_arr

In [ ]:
dashed=[]
lines=[]
for i in line_arr:
    r=stroke_features[i]
    length=r[7]
    if length<=80:
        dashed.append(i)
    else:
        lines.append(i)
dashed,lines

# Symbol Classification

In [ ]:
objects=[]
for i in obj_arr:
    objects.append(stroke_features[i])

df_obj=pd.DataFrame(objects,columns=['rf1','rf2','rf3','rf4','rf5','rf6','rf7','rf8','rf9','rf10','rf11','rf12','rf13'])
df_obj
model=pickle.load(open('RFC.sav','rb'))
y_pred=model.predict(df_obj)
y_pred
obj_dict={}
for i in range(len(obj_arr)):
    obj_dict[obj_arr[i]]=y_pred[i]
obj_dict

# Finding Circle in Square

In [ ]:
minma=[]
for i in obj_arr:
    segs=np.array(segments[i])
    min_vals=np.amin(segs,0)
    max_vals=np.amax(segs,0)
    minma.extend([[min_vals[1],max_vals[1],min_vals[2],max_vals[2]]])

circle_in_square=[]
for i in range(0,len(minma)):
    minmax2=minma[i]
    minmax1=minma
    for j in range(0,len(minmax1)):
        if j==i:
            pass
        else:
            rfj=minmax1[j]
            if abs(minmax2[0]-rfj[0])<100:
                if abs(minmax2[1]-rfj[1])<100:
                    if abs(minmax2[2]-rfj[2])<100:
                        if abs(minmax2[3]-rfj[3])<100:
                            if i not in circle_in_square:
                                circle_in_square.append(i)
                                circle_in_square.append(j)

In [ ]:
cont_objs=[]
for i in circle_in_square:
    cont_objs.append(obj_arr[i])
cont_objs

# Determining Association through Euclidean Distance

In [ ]:
sorted_neighbours=[]
for i in range(len(c)):
    a=c[i]
    xcalibre=np.argsort(a[1])
    xsorted=np.sort(a[1])
    xsorted=xsorted[xsorted<70.45]
    xsorted=xsorted[1:]
    xcalibre=xcalibre[1:len(xsorted)+1]
    sorted_neighbours.append(xcalibre)
sorted_neighbours

# Graph of connections

In [ ]:
lines_dict={}
for i in lines:
    lines_dict[i]='line'
dashed_dict={}
for i in dashed:
    dashed_dict[i]='dashed'

lines_dict.update(dashed_dict)
lines_dict.update(obj_dict)
lines_dict

In [ ]:
class Symbol():
    def __init__(self, symtype):
        self.symtype=symtype
        self.neighbors=set()
    def assoc(self, other):
        self.neighbors.add(other)
        other.neighbors.add(self)
    def dissoc(self, other):
        self.neighbors.discard(other)
        other.neighbors.discard(self)
    def merge(self, other=None):
        if len(self.neighbors)==0:
            return
        o=self.neighbors.pop()
        self.dissoc(o)
        d=self.neighbors.pop()
        self.dissoc(d)
        o.assoc(d)
    def __repr__(self):
        return self.symtype+' '+str([x.symtype for x in self.neighbors])

In [ ]:
symbols = [Symbol('None') for i in lines_dict]
for i in lines_dict.items():
    symbols[i[0]].symtype = i[1]
    for j in sorted_neighbours[i[0]]:
        symbols[i[0]].assoc(symbols[j])

In [ ]:
# Merges circle & square
s1, s2 = symbols[cont_objs[0]], symbols[cont_objs[1]]
s1.dissoc(s2)
s1.symtype += s2.symtype
s2.symtype = 'None'
s1.neighbors.update(s2.neighbors)
for i in symbols:
    i.dissoc(s2)

In [ ]:
# Remove line-dash association
temp = []
for i in symbols:
    for j in i.neighbors:
        if i.symtype == 'line' and j.symtype == 'dashed' or i.symtype == 'dashed' and j.symtype == 'line':
            temp.append((i, j))
for i in temp:
    i[0].dissoc(i[1])

In [ ]:
# Merge consecutive lines
for i in symbols:
    nlist=list(i.neighbors)
    for j in nlist:
        if (i.symtype=='line' or i.symtype=='dashed') and i.symtype==j.symtype:
            i.merge(j)

In [ ]:
#Merge Valve components
for i in symbols:
    temp=[x.symtype for x in i.neighbors]
    if i.symtype == 'Semi (' or 'Valve ' in temp and 'Semi (' in temp:
        i.merge()
symbols= [i for i in symbols if len(i.neighbors)>0]
symbols

# Graph amalgamation

In [ ]:
import schemdraw
from schemdraw.segments import *
from schemdraw.elements import *

In [ ]:
valHeight=1
valWidth=1
gap = (math.nan, math.nan)  # Put a gap in a path
class Valve1(Element):
    def __init__(self, *d, **kwargs):
        super().__init__(*d, **kwargs)
        self.segments.append(Segment([(0, 0),
                                      (-valWidth/2, -valHeight/2),
                                      (valWidth/2, -valHeight/2),
                                      (-valWidth/2, valHeight/2),
                                      (valWidth/2, valHeight/2),
                                      (0, 0)
                                      ]))
        self.segments.append(Segment([(0, valHeight/2),(0, valHeight/2+1)]))
        self.segments.append(Segment([(0, -valHeight/2),(0, -valHeight/2-1)]))
        self.segments.append(Segment([(0, 0),(1, 0)]))
        self.segments.append(Segment([(1, semiHeight),(1, -semiHeight)]))
        self.segments.append(SegmentArc((1, 0), semiWidth, semiWidth, 270, 90))
        self.anchors['start']=(1+semiWidth/2, 0)
        self.anchors['end']=(0,-valHeight/2)

ctrlHeight=0.5
ctrlWidth=1
class Control(Element2Term):
    ''' Generic source element '''
    def __init__(self, *d, **kwargs):
        super().__init__(*d, **kwargs)
        self.segments.append(Segment([(0, 0),gap,(ctrlWidth, 0)]))
        self.segments.append(Segment([(0, ctrlHeight),
                                      (0, -ctrlHeight),
                                      (ctrlWidth, -ctrlHeight),
                                      (ctrlWidth, ctrlHeight),
                                      (0, ctrlHeight)]))
        self.segments.append(SegmentCircle((0.5, 0), 0.5,))


semiHeight=0.5
semiWidth=1
class Semi(Element2Term):
    def __init__(self, *d, **kwargs):
        super().__init__(*d, **kwargs)
        self.segments.append(Segment([(0, 0),gap,(semiWidth, 0)]))
        self.segments.append(Segment([(0, semiHeight*2),(0, -semiHeight*2)]))
        self.segments.append(SegmentArc((0, 0), semiWidth*2, semiWidth*2, 270, 90))

class Circle(Element2Term):
    def __init__(self, *d, **kwargs):
        super().__init__(*d, **kwargs)
        self.segments.append(Segment([(0, 0), (0, 0), gap, (1, 0), (1, 0)]))
        self.segments.append(SegmentCircle((0.5, 0), 0.5,))

In [ ]:
import copy
backup=symbols

def drawSeg(sym):
    global theta
    if sym.symtype == 'line':
        # theta += 90
        return [Line(theta=theta)]
        # return Wire('-|', theta=theta)
        # return [Line()]
    if sym.symtype == 'dashed':
        # theta += 90
        return [Line(ls='--', theta=theta)]
        # return Wire('-|', ls='--', theta=theta)
        # return [Line(ls='--')]
    if sym.symtype == 'Circle':
        return [Circle(extend=False)]
    if sym.symtype == 'Valve ':
        theta=-90
        return [Valve1(extend=False).reverse()]
    if sym.symtype == 'Semi (':
        return [Semi(extend=False)]
    # if sym.symtype == 'SquareCircle':
    return [Control(extend=False)]


In [ ]:
def dfs(d, sym, endpoint):
    global btrack
    seg=drawSeg(sym)
    end=[endpoint]
    for i in seg:
        d+=(temp:= i.at(end[0]))
        endpoint=i.end
        btrack=i.end
        if sym.symtype=='Valve ':
            btrack= i.start
    print(sym.symtype, endpoint)
    ret=endpoint
    while len(sym.neighbors) > 0:
        neighbor=sym.neighbors.pop()
        sym.dissoc(neighbor)
        # print(sym.symtype, end, sym.neighbors)
        ret=dfs(d, neighbor, endpoint)
    return ret

In [ ]:
symbols = copy.deepcopy(backup)
theta = 0
with schemdraw.Drawing() as d:
    d.config(unit=2.0)
    endpoint = (0, 0)
    while len(symbols[0].neighbors)>0:
        symbols[0].dissoc(symbols[0].neighbors.pop())
    endpoint= dfs(d, symbols[1], endpoint)
    d+=Wire('-|').at(btrack).to((-1,0))
    d+=Line().to((0,0))